In [ ]:
import plotly.express as px
import numpy as np
import duckdb
from tqdm import tqdm

In [ ]:
import sys
sys.path.append("/home/ubuntu/sky_workdir/encoding-schemes")

from encoding_schemes import get_deterministic_adherence_fn

In [ ]:
import ray

ray.init()

In [ ]:
import os
import psycopg2
import json

conn_string = os.environ["SUPABASE_CONNECTION_URL"]

conn = psycopg2.connect(conn_string)

import pandas as pd

In [ ]:
# df = pd.read_sql("SELECT * FROM public.encoding_schemes WHERE (data->'experiment_tags'->'sft')::boolean", conn)

# sel_str = """
# -- redo prompted
# (
#     (data->'experiment_tags'->'numina_math_cot_rerun')::BOOL
#     AND (NOT (data->'force_overwrite')::BOOL OR data->'force_overwrite' IS NULL)
#     AND (data->'experiment_name')::TEXT LIKE '%prompted_%'
# )
# """

# sel_str = """
# -- Few shot
#  (
#      (data->'experiment_tags'->'numina_math_cot_rerun')::BOOL
#      AND (NOT (data->'force_overwrite')::BOOL OR data->'force_overwrite' IS NULL)
#      AND (
#          (data->'experiment_params'->'n_few_shot_examples')::INT = 8
#      )
#   )
# """

sel_str = """
-- NuminaMath CoT Rerun
 (
     (data->'experiment_tags'->'numina_math_cot_rerun')::BOOL
     AND (NOT (data->'force_overwrite')::BOOL OR data->'force_overwrite' IS NULL)
     AND (
         (data->'experiment_params'->'sampling_params'->'n')::INT = 4
         OR ((data->'experiment_params'->'model')::TEXT LIKE '%gpt%' AND (data->'experiment_params'->'sft_params'->'batch_size')::INT != 48)
     )
  )
"""

# sel_str = """
# -- NuminaMath CoT Rerun
#  (
#      (data->'experiment_tags'->'numina_math_cot_rerun')::BOOL
#      AND (NOT (data->'force_overwrite')::BOOL OR data->'force_overwrite' IS NULL)
#      AND (
#          (data->'experiment_params'->'model')::TEXT LIKE '%gpt%'
#      )
#      AND (
#         (data->'experiment_params'->'encoding_scheme')::TEXT LIKE '%reverse_letters%'
#         OR (data->'experiment_params'->'encoding_scheme')::TEXT LIKE '%identity%'
#      )
#   )
# """

# sel_str = """
# -- prompted no sft decode
# (
#     (data->'experiment_tags'->'numina_math_cot_rerun')::BOOL
#     AND (NOT (data->'force_overwrite')::BOOL OR data->'force_overwrite' IS NULL)
#     AND (data->'experiment_name')::TEXT LIKE '%prompteddecode%'
# )
# """

df = pd.read_sql(f"""
SELECT * FROM public.encoding_schemes 
    WHERE 

{sel_str}


ORDER BY created_at DESC
""", conn)

df.head()

In [ ]:
root_dir = "/home/ubuntu/sky_workdir/encoding-schemes/output"

In [ ]:
l_examples = df.to_dict('records')

l_examples[:5]

In [ ]:
[example for example in l_examples if example["data"]["experiment_params"]['encoding_scheme'] == 'speaking_base64_cipher' and "gpt-4.1-2025" in example["data"]["experiment_params"]["model"]]

# Debugging

In [ ]:
l_acc = []

for temp in [0.001, 0.1, 0.3, 0.5, 0.7, 1.0]:
    df_fewshot = pd.read_parquet(f"/home/ubuntu/sky_workdir/encoding-schemes/output/27888d7de3b05cca5d2645c21cc83ffda26650ee/data/temperature_{temp}/math_scores.parquet") 


    l_acc.append(np.mean(df_fewshot['is_corrects'].tolist()).flatten())

l_acc, [0.001, 0.1, 0.3, 0.5, 0.7, 1.0]

In [ ]:
fig = px.line(y=np.array(l_acc).flatten(), x=[0.001, 0.1, 0.3, 0.5, 0.7, 1.0])

fig.update_yaxes(title="MATH 500 accuracy")
fig.update_xaxes(title="temperature")
fig.update_layout(height=600, width=800, title="Temperature sweep for Qwen2.5 14B, reverse letters in each word")

fig.show()

In [ ]:
from datasets import load_dataset

ds = load_dataset("GAIR/LIMO")

ds['train']

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-14B-Instruct")

n_total_tok = 0

for row in ds['train']:
    n_total_tok += len(tokenizer.encode(row['question']))
    n_total_tok += len(tokenizer.encode(row['solution']))

n_total_tok

In [ ]:
from datasets import load_dataset

ds = load_dataset("simplescaling/s1K")

ds['train']

In [ ]:
n_total_tok = 0

for row in ds['train']:
    n_total_tok += len(tokenizer.encode(row['question']))
    n_total_tok += len(tokenizer.encode(row['thinking_trajectories'][0]))

n_total_tok

In [ ]:
df_fewshot = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/6bedc631358628ba4327951c3ab1753788e886bf/data/exit0/math_scores.parquet")
np.mean(list(df_fewshot['is_corrects']))

In [ ]:
df_fewshot = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/6bedc631358628ba4327951c3ab1753788e886bf/data/exit1/math_scores.parquet")
np.mean(list(df_fewshot['is_corrects']))

In [ ]:
df_fewshot = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/6bedc631358628ba4327951c3ab1753788e886bf/data/exit2/math_scores.parquet")
np.mean(list(df_fewshot['is_corrects']))

In [ ]:
df_fewshot = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/6bedc631358628ba4327951c3ab1753788e886bf/data/final/math_scores.parquet")
np.mean(list(df_fewshot['is_corrects']))

In [ ]:
# df_fewshot = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/66525cd7fd3d441670d728733520fec9f16a8dd0/data/joined_output.parquet") # rot13
# df_fewshot = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/fa6964749b4f5e04200981cf46cc85a15b22cdfc/data/joined_output.parquet") # reverse letters in each word
# df_fewshot = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/7b88a15cc7e51bd9ce792e47c494d2530577f989/data/joined_output.parquet") # base64
# df_fewshot = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/19a13b8d5227b89e30470dfbec5588029e6cbb8a/data/joined_output.parquet") # identity sonnet 35
# df_fewshot = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/d3bd4346e68ace0f3ea5eb3d0a05730e42ea2281/data/length_control/math_scores.parquet") 
# df_fewshot = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/d3bd4346e68ace0f3ea5eb3d0a05730e42ea2281/data/math_scores.parquet") 
# df_fewshot = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/7b350b992b24426e6c5b9de2892e29b4736dc730/data/final/math_scores.parquet")
df_fewshot = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/6bedc631358628ba4327951c3ab1753788e886bf/data/final/math_scores.parquet")


df_fewshot.head()

In [ ]:
from encoding_schemes.letter_permutations import reverse_letters_in_each_word

In [ ]:
np.mean(list(df_fewshot['is_corrects']))

In [ ]:
reverse_letters_in_each_word(df_fewshot['model_cot'].iloc[8][0])

In [ ]:
reverse_letters_in_each_word(df_fewshot['model_cot'].iloc[1][0])

In [ ]:
df_sft = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/7b88a15cc7e51bd9ce792e47c494d2530577f989/data/sft_train.parquet") # base64

len(df_sft), df_sft['num_tokens'].sum()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-14B-Instruct")


In [ ]:
# tokenize & get output lens

df_fewshot['n_cot_tokens'] = df_fewshot['model_cot'].map(lambda x: [len(tokenizer.encode(s)) for s in x])

In [ ]:
l_tokens = np.array(df_fewshot['n_cot_tokens'].tolist()).flatten()
l_is_correct = np.array(df_fewshot['is_corrects'].tolist()).flatten()

In [ ]:
len(l_tokens), len(l_is_correct)

In [ ]:
l_token_buckets = [int(n_tok // 200) * 200 for n_tok in l_tokens]

In [ ]:
df_query = pd.DataFrame({'is_correct': l_is_correct, 'n_tokens': l_tokens})

df_query = duckdb.query("SELECT CAST(n_tokens / 200 AS INT) * 200 as n_token_bucket, AVG(is_correct) as is_correct_pct, COUNT(*) as ct FROM df_query WHERE n_token_bucket < 10000 GROUP BY n_token_bucket HAVING COUNT(*) > 50").to_df()

fig = px.bar(df_query, x='n_token_bucket', y='is_correct_pct', width=800, height=600, title="Length sweep, reverse_letters_in_each_word_no_math_expressions")#, color='ct')

fig.update_xaxes(dtick=200, title="# tokens (bucket)")
fig.update_yaxes(dtick=0.05, title="MATH 500 accuracy")

fig.show()

In [ ]:
l_difficulty_controlled = []

for _, row in df_fewshot.iterrows():
    sorted_indices = np.argsort(row['n_cot_tokens'])
    for i, idx in enumerate(sorted_indices):
        l_difficulty_controlled.append({
            'n_tokens': row['n_cot_tokens'][idx],
            'bucket': i,
            'is_correct': row['is_corrects'][idx]
        })
df_difficulty_controlled = pd.DataFrame(l_difficulty_controlled)

In [ ]:
df_query = duckdb.query("SELECT bucket, AVG(is_correct) as is_correct_pct, COUNT(*) as ct FROM df_difficulty_controlled GROUP BY bucket HAVING COUNT(*) > 50").to_df()

fig = px.bar(df_query, x='bucket', y='is_correct_pct', width=800, height=600, title="Effect of length on math accuracy, reverse_letters_in_each_word_no_math_expressions")#, color='ct')

fig.update_xaxes(dtick=1, title="# tokens (bucket ordered by token length)")
fig.update_yaxes(dtick=0.05, title="MATH 500 accuracy")

fig.show()

In [ ]:
df_query_reverse = df_difficulty_controlled.copy()
df_query_reverse['encoding_scheme'] = 'reverse_letters_in_each_word_no_math_expressions'

In [ ]:
df_fewshot_identity = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/dd7f9ee4bb4c09a61f663823c4a62158fcda90b5/data/length_control/math_scores.parquet") 

df_fewshot_identity.head()

In [ ]:
df_fewshot_identity['n_cot_tokens'] = df_fewshot_identity['model_cot'].map(lambda x: [len(tokenizer.encode(s)) for s in x])

l_difficulty_controlled = []

for _, row in df_fewshot_identity.iterrows():
    sorted_indices = np.argsort(row['n_cot_tokens'])
    for i, idx in enumerate(sorted_indices):
        l_difficulty_controlled.append({
            'n_tokens': row['n_cot_tokens'][idx],
            'bucket': i,
            'is_correct': row['is_corrects'][idx]
        })
df_difficulty_controlled = pd.DataFrame(l_difficulty_controlled)

In [ ]:
l_tokens = np.array(df_fewshot_identity['n_cot_tokens'].tolist()).flatten()
l_is_correct = np.array(df_fewshot_identity['is_corrects'].tolist()).flatten()

df_query = pd.DataFrame({'is_correct': l_is_correct, 'n_tokens': l_tokens})

df_query = duckdb.query("SELECT CAST(n_tokens / 200 AS INT) * 200 as n_token_bucket, AVG(is_correct) as is_correct_pct, COUNT(*) as ct FROM df_query WHERE n_token_bucket < 10000 GROUP BY n_token_bucket HAVING COUNT(*) > 50").to_df()

fig = px.bar(df_query, x='n_token_bucket', y='is_correct_pct', width=800, height=600, title="Length sweep, identity")#, color='ct')

fig.update_xaxes(dtick=200, title="# tokens (bucket)")
fig.update_yaxes(dtick=0.05, title="MATH 500 accuracy")

fig.show()

In [ ]:
df_query = duckdb.query("SELECT bucket, AVG(is_correct) as is_correct_pct, COUNT(*) as ct FROM df_difficulty_controlled GROUP BY bucket HAVING COUNT(*) > 50").to_df()

fig = px.bar(df_query, x='bucket', y='is_correct_pct', width=800, height=600, title="Effect of length on math accuracy, identity")#, color='ct')

fig.update_xaxes(dtick=1, title="# tokens (bucket ordered by token length)")
fig.update_yaxes(dtick=0.05, title="MATH 500 accuracy")

fig.show()

In [ ]:
df_query_identity = df_difficulty_controlled.copy()
df_query_identity['encoding_scheme'] = 'identity'

In [ ]:
df_difficulty_controlled_combined = pd.concat([df_query_identity, df_query_reverse], ignore_index=True)


In [ ]:
df_difficulty_controlled_combined.head()

In [ ]:
df_query = duckdb.query("SELECT CAST(n_tokens / 200 AS INT) * 200 as n_token_bucket, encoding_scheme, AVG(is_correct) as is_correct_pct, COUNT(*) as ct FROM df_difficulty_controlled_combined WHERE n_token_bucket < 10000 GROUP BY n_token_bucket, encoding_scheme HAVING COUNT(*) > 50 ORDER BY n_token_bucket, encoding_scheme").to_df()

fig = px.line(df_query, x='n_token_bucket', y='is_correct_pct', width=1000, height=600, title="Token length vs MATH accuracy", color="encoding_scheme")#, barmode="group")#, color='ct')

fig.update_xaxes(dtick=200, title="# tokens (bucket)")
fig.update_yaxes(dtick=0.05, title="MATH 500 accuracy")

fig.show()

In [ ]:

df_query = duckdb.query("""SELECT bucket, encoding_scheme, AVG(is_correct) as is_correct_pct, 
AVG(is_correct) / (SELECT AVG(is_correct) FROM df_difficulty_controlled_combined a WHERE a.bucket = 0 AND b.encoding_scheme = a.encoding_scheme) as rel_acc_to_bucket_0,
COUNT(*) as ct,
AVG(n_tokens) as n_tok
FROM df_difficulty_controlled_combined b GROUP BY bucket, encoding_scheme HAVING COUNT(*) > 50
ORDER BY bucket, encoding_scheme
""").to_df()

fig = px.line(df_query, x='bucket', y='rel_acc_to_bucket_0', color='encoding_scheme', width=1000, height=600, title="Effect of length on math accuracy")
            # barmode='group')#, color='ct')

fig.update_xaxes(dtick=1, title="Rollout bucket (ordered by length)", tickangle=0)
fig.update_yaxes(dtick=0.05, title="Accuracy relative to shortest rollout per problem", tickangle=0)

fig.show()

In [ ]:
fig = px.line(df_query, x='n_tok', y='bucket', color='encoding_scheme', width=1000, height=600, title="Bucket length differences")
            # barmode='group')#, color='ct')

fig.update_xaxes(title="mean # tokens in bucket", tickangle=0)
fig.update_yaxes(dtick=1, title="Rollout bucket (ordered by length)", tickangle=0)

fig.show()

In [ ]:
test_idx = 4

In [ ]:
df_fewshot.iloc[test_idx]['backtranslation_prompt']

In [ ]:
df_fewshot.iloc[test_idx]['target_backtranslation_text']

In [ ]:
df_fewshot.iloc[test_idx]['generated_backtranslations'][0]

# Code

In [ ]:

def bootstrap_ci(data, statistic=np.mean, alpha=0.05, n_boot=10_000, random_state=None):
    """
    Returns (point_estimate, low_CI, high_CI) for given 1D data.
    Works with bool, int, or float data.
    """
    x = np.asarray(data).astype(float)  # ensure numeric
    x = x[~np.isnan(x)]
    if len(x) == 0:
        raise ValueError("No valid data for bootstrapping.")

    rng = np.random.default_rng(random_state)
    n = len(x)

    # Draw bootstrap samples
    idx = rng.integers(0, n, size=(n_boot, n))
    samples = x[idx]

    # Apply statistic row-wise
    stats = np.apply_along_axis(statistic, 1, samples)

    point = statistic(x)
    lo = np.percentile(stats, 100 * (alpha / 2))
    hi = np.percentile(stats, 100 * (1 - alpha / 2))
    return point, lo, hi

In [ ]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")


@ray.remote
def count_tokens_from_messages(s):
    try:
        return len(encoding.encode(s, disallowed_special=()))
    except ValueError as e:
        print(e)
        return 0


In [ ]:
@ray.remote(num_cpus=2, memory=32 * 1024 * 1024 * 1024)
def compute_translation_token_count(example, df_data):
    sys.path.append("/home/ubuntu/sky_workdir/encoding-schemes")

    from orchestration.experiment_meta_saver import compute_experiment_hash
    from utils.io_utils import read_large_parquet

    from transformers import AutoTokenizer

    model = example["data"]["experiment_params"]["model"]
    if "gpt" in model or "claude" in model:
        print(f"Overriding tokenizer for {model} with gpt-oss 120b tokenizer because it was detected as a GPT/Claude model!")
        model = "openai/gpt-oss-120b"

    tokenizer = AutoTokenizer.from_pretrained(model)

    return df_data['reference_solution'].map(lambda x: len(tokenizer.encode(x)))


In [ ]:
from tqdm import tqdm

In [ ]:
def compute_ci_cols(example, df, col_name, transformation_fn):
    s_transformed = transformation_fn(df[col_name])
    if np.isscalar(s_transformed) and np.isnan(s_transformed):
        print(f"Warning: {col_name} was all NaN, ignoring!")
        return
    mid, lo, hi = bootstrap_ci(s_transformed)
    example[col_name] = mid
    example[f'{col_name}_low_ci'] = mid - lo
    example[f'{col_name}_hi_ci'] = hi - mid

In [ ]:
def compute_multi_row_transformation(df, row_transform_fn, col_name, agg_fn):
    df[col_name] = df.apply(row_transform_fn, axis=1)

In [ ]:

def _score_rollouts(rollouts):
    # rollouts: list/array of rollout sequences; may also be None/np.nan/scalar
    if rollouts is None or (isinstance(rollouts, float) and np.isnan(rollouts)):
        return np.nan

    vals = []
    for r in rollouts:
        # skip None/NaN
        if r is None or (isinstance(r, float) and np.isnan(r)):
            continue
        vals.append(-np.nansum(r))
    return np.nanmean(vals) if len(vals) else np.nan


def patch_gpt_api_log_loss(example):
    experiment_hash = example['experiment_hash']

    translation_loss = os.path.join(root_dir, experiment_hash, "data", f"validation_reverse_translation_math500_meta.json")
    with open(translation_loss, "r") as fp:
        example["backtranslation_gt_logprobs"] = translation_loss["valid_loss"]

    # need to be validation loss on 512k...
    validation_loss = os.path.join(root_dir, experiment_hash, "data", f"validation_reverse_translation_math500_meta.json")
    with open(validation_loss, "r") as fp:
        example["backtranslation_gt_logprobs"] = translation_loss["valid_loss"]


@ray.remote
def process_single_example(example):
    experiment_hash = example['experiment_hash']
    
    target_path = os.path.join(root_dir, example['experiment_hash'], "data", "joined_output.parquet")
    if not os.path.exists(target_path):
        print(f"!!!!!! {target_path} missing !!!!!!!")
        return example
    
    df_data = pd.read_parquet(target_path)

    d_col_to_transform = {
        'cot_gt_logprobs' : lambda s: np.nansum(s.map(_score_rollouts)),
        'generated_cot_is_correct' : np.mean,  # was np.mean
        'backtranslation_gt_logprobs' : lambda s: np.nanmean(s.map(_score_rollouts)),
        'backtranslation_bleu_scores' : np.mean,  # was np.mean
        'generated_cot_adhered_encoding_style': np.mean  # was np.mean
    }
    for col, fn in d_col_to_transform.items():
        if col not in df_data:
            print(col)
            print(df_data.head())
            print(example)
            raise Exception(str(col) + "\n" + str(df_data.head()) + "\n" + str(example))

        compute_ci_cols(example, df_data, col, fn)

    df_data["num_tokens_translation_output"] = ray.get(compute_translation_token_count.remote(example, df_data))

    d_and_cols = {
        'adherent_and_correct': (
            lambda r: np.nanmean( \
                np.array(r['generated_cot_is_correct']).astype(bool) & \
                np.array(r['generated_cot_adhered_encoding_style']).astype(bool) \
            ),
            np.nanmean
        ),
        'total_translation_loss': (
            lambda r: np.nanmean( \
                np.array(r['num_tokens_translation_output']) * \
                np.array(np.nanmean(_score_rollouts(r['backtranslation_gt_logprobs'])), dtype=np.float64) \
            ),
            np.nanmean
        ),
    }
    for col, (transform_fn, agg_fn) in d_and_cols.items():
        compute_multi_row_transformation(df_data, transform_fn, col, agg_fn)
        if df_data[col].isna().sum() != len(df_data):
            compute_ci_cols(example, df_data, col, lambda x: x)

    if "gpt" in example["data"]["experiment_params"]["model"] and "use_api_sft_model_for_sampling" in example["data"]["experiment_params"]:
        with open(os.path.join(root_dir, example['experiment_hash'], "data", f"validation_reverse_translation_math500_meta.json"), "r") as fp:
            d_model_meta = json.load(fp)

        example["backtranslation_gt_logprobs"] = d_model_meta["valid_loss"]
        example["total_translation_loss"] = d_model_meta["valid_loss"] * np.nanmean(df_data["num_tokens_translation_output"])
        example["total_translation_loss_low_ci"] = 0.0
        example["total_translation_loss_hi_ci"] = 0.0

    pretraining_prevalence_path = os.path.join('/home/ubuntu/sky_workdir/encoding-schemes', 'output', experiment_hash, 'data', 'num_pretraining_4grams_redpajama.json')
    if os.path.exists(pretraining_prevalence_path):
        with open(pretraining_prevalence_path, "r") as fp:
            d_pretraining_prevalence = json.load(fp)

        example["pretraining_prevalence"] = d_pretraining_prevalence["num_occurrences"]


    for temp in [0.001, 0.1, 0.3, 0.5, 0.7, 1.0]:
        df_temp = f"/home/ubuntu/sky_workdir/encoding-schemes/output/{experiment_hash}/data/temperature_{temp}/math_scores.parquet"
        if os.path.exists(df_temp):
            df_temp = pd.read_parquet(df_temp)

            if temp == 0.001:
                temp = 0.0
            example[f"temp_{temp}_accuracy"] = np.mean(list(df_temp['is_corrects']))

    for col in df_data.columns:
        example[f"{col}_df"] = df_data[col]

    return example


l_new_examples = [None for _ in range(len(l_examples))]

for i, example in tqdm(enumerate(l_examples)):
    # l_examples[i] = process_single_example(example)
    l_new_examples[i] = process_single_example.remote(example)

for i, example in tqdm(enumerate(l_new_examples)):
    try:
        l_new_examples[i] = ray.get(example)
    except ray.exceptions.RayTaskError as e:
        l_new_examples[i] = l_examples[i]
        print(e)

l_examples = l_new_examples

In [ ]:
def humanize_number(num: float) -> str:
    """
    Converts a number into a human-readable string with k, M, or B suffixes.
    
    Args:
        num (float): The number to format.
    
    Returns:
        str: Human-readable string representation.
    """
    if num >= 1_000_000_000:
        return f"{num / 1_000_000_000:.1f}B"
    elif num >= 1_000_000:
        return f"{num / 1_000_000:.1f}M"
    elif num >= 1_000:
        return f"{num / 1_000:.1f}k"
    else:
        return str(num)

In [ ]:
import re

def parse_params(model):
    if 'gpt' in model:
        if 'nano' in model:
            return 0
        elif 'mini' in model:
            return 1
        else:
            return 2


    if 'claude' in model:
        if 'haiku' in model:
            return 3
        elif 'sonnet' in model:
            return 4
        else:
            return 5
    
    return int(re.search("([0-9]+)B", model).group(1))

In [ ]:
df_viz = pd.DataFrame(l_examples)

orig_len = len(df_viz)

# df_viz = df_viz[df_viz['cot_gt_logprobs'].notna()]

new_len = len(df_viz)
if orig_len != new_len:
    print(f"Lost {orig_len - new_len} examples from na logprobs")

df_viz['encoding_scheme'] = df_viz['data'].map(lambda x: x['experiment_params']['encoding_scheme'])
df_viz['model'] = df_viz['data'].map(lambda x: x['experiment_params']['model'])

try:
    df_viz['model_size'] = df_viz['model'].map(parse_params)
except Exception as e:
    print(e)
df_viz['input_type'] = df_viz['data'].map(lambda x: "_".join(x['experiment_name'].split("_")[:2]))

df_viz['n_few_shot_examples'] = df_viz['data'].map(lambda x: x['experiment_params'].get('n_few_shot_examples', None))

df_viz['Adherence Calculation Method'] = df_viz['encoding_scheme'].map(lambda x: get_deterministic_adherence_fn(x, None) is not None).map({ True: 'deterministic', False: 'Sonnet 4 judge'})

try:
    df_viz['total_train_tok'] = df_viz['n_total_train_tok'].map(humanize_number)
except Exception as e:
    print(e)

df_viz.head()

In [ ]:
df_viz['input_type'].unique()

In [ ]:
# filter_set = ['mathcot_fewshot']
filter_set = ['math_cot']
# filter_set = ['math_cot', 'mathcot_fewshot']
# filter_set = ['mathcot_prompted']
# filter_set = ['mathcot_prompteddecode']

In [ ]:
df_viz_tmp = df_viz[df_viz['input_type'].isin(filter_set)]
df_viz_tmp = df_viz_tmp[df_viz_tmp['model'] != 'Qwen/Qwen2.5-7B']

df_viz_tmp = df_viz_tmp.sort_values([
    'model_size',
    'adherent_and_correct'
])

df_viz_tmp = df_viz_tmp.astype({'n_few_shot_examples': str})

df_viz_tmp['encoding_scheme'] = df_viz_tmp['encoding_scheme'].map(lambda s: s.split("speaking_")[-1])

In [ ]:
df_viz_tmp[df_viz_tmp['model'].str.contains('gpt')]['encoding_scheme'].unique()

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

# Melt the dataframe
df_plot = pd.melt(df_viz_tmp, 
                  id_vars=['encoding_scheme', 'model'], 
                  value_vars=[f"temp_{temp}_accuracy" for temp in [0.0, 0.1, 0.3, 0.5, 0.7, 1.0]])

df_plot = df_plot[df_plot['value'].notna()]
df_plot = df_plot[df_plot['model'].str.contains('14B')]

# Create a color map for the temperature values
temps = [0.0, 0.1, 0.3, 0.5, 0.7, 1.0]
# Create colors from a continuous colormap
import plotly.colors as pc
colors = pc.sample_colorscale('Plasma', np.linspace(0, 1, len(temps)))

# Map variable names to colors
color_discrete_map = {f"temp_{temp}_accuracy": colors[i] for i, temp in enumerate(temps)}

# Create the bar chart
fig = px.bar(df_plot, 
             x='encoding_scheme', 
             color='variable', 
             y='value', 
             barmode='group',
             color_discrete_map=color_discrete_map,
             labels={'value': 'Accuracy relative to identity', 
                     'variable': 'Temperature',
                     'encoding_scheme': 'Encoding Scheme'})

# Update legend labels to show just temperature values
fig.for_each_trace(lambda t: t.update(name=t.name.replace('temp_', 'T=').replace('_accuracy', '')))

fig.update_layout(title='Accuracy Relative to Identity by Encoding Scheme and Temperature, Qwen2.5 14B')

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

d_mapping = {}
for _, row in df_viz_tmp.iterrows():
    for temp in [0.0, 0.1, 0.3, 0.5, 0.7, 1.0]:
        d_mapping[(row['model'], row['encoding_scheme'], temp)] = df_viz_tmp[(df_viz_tmp['model'] == row['model']) & (df_viz_tmp['encoding_scheme'] == 'identity')][f"temp_{temp}_accuracy"].iloc[0]

for temp in [0.0, 0.1, 0.3, 0.5, 0.7, 1.0]:
    df_viz_tmp[f"temp_{temp}_rel_acc"] = df_viz_tmp.apply(lambda r: r[f"temp_{temp}_accuracy"] / d_mapping[(r['model'], r['encoding_scheme'], temp)], axis=1)

# Melt the dataframe
df_plot = pd.melt(df_viz_tmp, 
                  id_vars=['encoding_scheme', 'model'], 
                  value_vars=[f"temp_{temp}_rel_acc" for temp in [0.0, 0.1, 0.3, 0.5, 0.7, 1.0]])

df_plot = df_plot[df_plot['value'].notna()]
df_plot = df_plot[df_plot['model'].str.contains('14B')]

# Create a color map for the temperature values
temps = [0.0, 0.1, 0.3, 0.5, 0.7, 1.0]
# Create colors from a continuous colormap
import plotly.colors as pc
colors = pc.sample_colorscale('Plasma', np.linspace(0, 1, len(temps)))

# Map variable names to colors
color_discrete_map = {f"temp_{temp}_rel_acc": colors[i] for i, temp in enumerate(temps)}

# Create the bar chart
fig = px.bar(df_plot, 
             x='encoding_scheme', 
             color='variable', 
             y='value', 
             barmode='group',
             color_discrete_map=color_discrete_map,
             labels={'value': 'Accuracy relative to identity', 
                     'variable': 'Temperature',
                     'encoding_scheme': 'Encoding Scheme'})

# Update legend labels to show just temperature values
fig.for_each_trace(lambda t: t.update(name=t.name.replace('temp_', 'T=').replace('_rel_acc', '')))

fig.update_layout(title='Accuracy Relative to Identity by Encoding Scheme and Temperature, Qwen2.5 14B')

fig.show()


# Adherence plot

In [ ]:
df_viz_tmp_plot = df_viz_tmp.copy()

fig = px.bar(df_viz_tmp_plot, x='encoding_scheme', y='generated_cot_adhered_encoding_style',
             height=800, width=1600,
             # height=1600, width=1600,
             # color='model',
             error_y='generated_cot_adhered_encoding_style_hi_ci',
             error_y_minus='generated_cot_adhered_encoding_style_low_ci',
             # color='n_few_shot_examples',
             # facet_row='model',
             color='model',
             facet_col='Adherence Calculation Method',
             # color='training_augmentation',
             # color='total_train_tok',
             # color_discrete_map=color_discrete_map,
             title="MATH 500 CoT encoding style adherence",
             barmode="group",
             template="plotly_white",
             color_discrete_sequence=px.colors.qualitative.Set2,
            )

fig.update_xaxes(title="Encoding scheme", tickangle=90)
fig.update_yaxes(title="% adherent encodings", dtick=0.1)

# ✅ Make each facet's x-axis independent to avoid empty slots
fig.for_each_xaxis(lambda ax: ax.update(matches=None, categoryorder='trace'))

# ✅ Align all x-axis titles by setting a fixed vertical offset
ct = [0]
def ax_standoff_updater(ax, ct):    
    ax.title.update(standoff=ct[0] * 130)
    ct[0] += 1

fig.for_each_xaxis(lambda ax: ax_standoff_updater(ax, ct))
fig.update_yaxes(title_standoff=5)

fig.show()

# Pretraining prevalence plot

In [ ]:
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import plotly.graph_objects as go
import plotly.express as px

df_viz_tmp_plot = df_viz_tmp.copy()
df_viz_tmp_plot['encoding_scheme'] = df_viz_tmp_plot['encoding_scheme'].str.replace('_', ' ')

l_algo_encodings = [
    'base64 cipher',
    'base64 2x cipher',
    'base64 3x cipher',
    'gzip to base64 encoded',
    'letter to word with dot',
    'dot between chars',
    'space between chars'
]

# Separate data into two groups
df_l_algo = df_viz_tmp_plot[df_viz_tmp_plot['encoding_scheme'].isin(l_algo_encodings)]
df_rest = df_viz_tmp_plot[~df_viz_tmp_plot['encoding_scheme'].isin(l_algo_encodings)]

# Create the scatter plot with color coding
df_viz_tmp_plot['group'] = df_viz_tmp_plot['encoding_scheme'].apply(
    lambda x: 'Algorithmic' if x in l_algo_encodings else 'Language'
)

fig = px.scatter(df_viz_tmp_plot, x='pretraining_prevalence', y='adherent_and_correct', 
                 log_x=True, text='encoding_scheme', color='group',
                 color_discrete_map={'Algorithmic': 'red', 'Language': 'blue'})

# Update the scatter trace to not be part of the legend and keep its original appearance
fig.update_traces(textposition="top center", showlegend=False, selector=dict(mode='markers+text'))
fig.update_layout(legend_title_text=None) 

# Define fit functions
def linear_func(x, a, b):
    """Linear function: y = ax + b"""
    return a * x + b

def exponential_func(x, a, b, c):
    """Exponential function: y = a * exp(b * x) + c"""
    return a * np.exp(b * x) + c

def sigmoid_func(x, a, b, c, d):
    """Sigmoid function: y = a / (1 + exp(-b * (x - c))) + d"""
    return a / (1 + np.exp(-b * (x - c))) + d

def calculate_fit(df, fit_type='linear', use_log_x=True):
    """
    Calculate trend line based on specified fit type.
    
    Parameters:
    - df: DataFrame with data
    - fit_type: 'linear', 'exponential', or 'sigmoid'
    - use_log_x: Whether to use log-transformed x values
    """
    # Remove any rows with NaN or zero/negative x values
    df_clean = df.dropna(subset=['pretraining_prevalence', 'generated_cot_is_correct'])
    df_clean = df_clean[df_clean['pretraining_prevalence'] > 0]
    
    if len(df_clean) < 2:
        return None, None, None
    
    # Prepare x and y values
    if use_log_x:
        x = np.log10(df_clean['pretraining_prevalence'].values)
    else:
        x = df_clean['pretraining_prevalence'].values
    y = df_clean['adherent_and_correct'].values
    
    try:
        if fit_type == 'linear':
            # Linear regression
            slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
            params = [slope, intercept]
            
            # Calculate predictions for R²
            y_pred = linear_func(x, *params)
            
        elif fit_type == 'exponential':
            # Exponential fit
            # Initial guess: [amplitude, growth_rate, offset]
            initial_guess = [0.5, 0.1, 0.3]
            bounds = ([0, -np.inf, 0], [1, np.inf, 1])
            params, _ = curve_fit(exponential_func, x, y, p0=initial_guess, 
                                bounds=bounds, maxfev=5000)
            
            # Calculate predictions for R²
            y_pred = exponential_func(x, *params)
            
        elif fit_type == 'sigmoid':
            # Sigmoid fit
            # Initial guess: [amplitude, steepness, x_mid, y_offset]
            x_mid = np.median(x)
            initial_guess = [0.7, 1.0, x_mid, 0.2]
            bounds = ([0, 0, x.min(), 0], [1, 10, x.max(), 1])
            params, _ = curve_fit(sigmoid_func, x, y, p0=initial_guess, 
                                bounds=bounds, maxfev=5000)
            
            # Calculate predictions for R²
            y_pred = sigmoid_func(x, *params)
            
        else:
            raise ValueError(f"Unknown fit_type: {fit_type}")
        
        # Calculate R²
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        r_squared = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        
        return params, r_squared, fit_type
        
    except Exception as e:
        print(f"Error fitting {fit_type}: {e}")
        return None, None, None

# Configuration - CHANGE THESE TO SELECT FIT TYPE
FIT_TYPE = 'linear'  # Options: 'linear', 'exponential', 'sigmoid'
USE_LOG_X = False  # Whether to use log-transformed x for fitting

# Calculate trends for both groups
params_l_algo, r2_l_algo, _ = calculate_fit(df_l_algo, fit_type=FIT_TYPE, use_log_x=USE_LOG_X)
params_rest, r2_rest, _ = calculate_fit(df_rest, fit_type=FIT_TYPE, use_log_x=USE_LOG_X)

# Generate trend line points
x_range = df_viz_tmp_plot['pretraining_prevalence'].values
x_range = x_range[x_range > 0]  # Remove non-positive values
x_range = np.sort(x_range)

if USE_LOG_X:
    x_fit_range = np.log10(x_range)
else:
    x_fit_range = x_range

# Add trend lines if calculations were successful
if params_l_algo is not None:
    if FIT_TYPE == 'linear':
        y_trend_l_algo = linear_func(x_fit_range, *params_l_algo)
    elif FIT_TYPE == 'exponential':
        y_trend_l_algo = exponential_func(x_fit_range, *params_l_algo)
    elif FIT_TYPE == 'sigmoid':
        y_trend_l_algo = sigmoid_func(x_fit_range, *params_l_algo)
    
    fig.add_trace(go.Scatter(
        x=x_range,
        y=y_trend_l_algo,
        mode='lines',
        name=f'Algorithmic encoding, {FIT_TYPE} fit (R²={r2_l_algo:.3f})',
        line=dict(color='red', dash='dash'),
        showlegend=True
    ))

if params_rest is not None:
    if FIT_TYPE == 'linear':
        y_trend_rest = linear_func(x_fit_range, *params_rest)
    elif FIT_TYPE == 'exponential':
        y_trend_rest = exponential_func(x_fit_range, *params_rest)
    elif FIT_TYPE == 'sigmoid':
        y_trend_rest = sigmoid_func(x_fit_range, *params_rest)
    
    fig.add_trace(go.Scatter(
        x=x_range,
        y=y_trend_rest,
        mode='lines',
        name=f'Language encoding, {FIT_TYPE} fit (R²={r2_rest:.3f})',
        line=dict(color='blue', dash='dash'),
        showlegend=True
    ))

# Update layout
fig.update_layout(
    height=600, 
    width=800,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    # title=f"MATH500 Accuracy vs Pretraining Prevalence ({FIT_TYPE.capitalize()} Fit)"
)

fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))

fig.update_yaxes(dtick=0.1, title="% adherent & correct responses", range=[0, 0.6])
fig.update_xaxes(title="Pretraining prevalence", type="log")
fig.update_layout(template="plotly_white")
fig.update_traces(textfont_size=10)

fig.show('png')


In [ ]:
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import plotly.graph_objects as go
import plotly.express as px

df_viz_tmp_plot = df_viz_tmp.copy()
df_viz_tmp_plot['encoding_scheme'] = df_viz_tmp_plot['encoding_scheme'].str.replace('_', ' ')

l_algo_encodings = [
    'base64 cipher',
    'base64 2x cipher',
    'base64 3x cipher',
    'gzip to base64 encoded',
    'letter to word with dot',
    'dot between chars',
    'space between chars'
]

# Separate data into two groups
df_l_algo = df_viz_tmp_plot[df_viz_tmp_plot['encoding_scheme'].isin(l_algo_encodings)]
df_rest = df_viz_tmp_plot[~df_viz_tmp_plot['encoding_scheme'].isin(l_algo_encodings)]

# Create the scatter plot with color coding
df_viz_tmp_plot['group'] = df_viz_tmp_plot['encoding_scheme'].apply(
    lambda x: 'Algorithmic' if x in l_algo_encodings else 'Language'
)

fig = px.scatter(df_viz_tmp_plot, x='pretraining_prevalence', y='generated_cot_is_correct', 
                 log_x=True, text='encoding_scheme', color='group',
                 color_discrete_map={'Algorithmic': 'red', 'Language': 'blue'})

# Update the scatter trace to not be part of the legend and keep its original appearance
fig.update_traces(textposition="top center", showlegend=False, selector=dict(mode='markers+text'))
fig.update_layout(legend_title_text=None) 

# Define fit functions
def linear_func(x, a, b):
    """Linear function: y = ax + b"""
    return a * x + b

def exponential_func(x, a, b, c):
    """Exponential function: y = a * exp(b * x) + c"""
    return a * np.exp(b * x) + c

def sigmoid_func(x, a, b, c, d):
    """Sigmoid function: y = a / (1 + exp(-b * (x - c))) + d"""
    return a / (1 + np.exp(-b * (x - c))) + d

def calculate_fit(df, fit_type='linear', use_log_x=True):
    """
    Calculate trend line based on specified fit type.
    
    Parameters:
    - df: DataFrame with data
    - fit_type: 'linear', 'exponential', or 'sigmoid'
    - use_log_x: Whether to use log-transformed x values
    """
    # Remove any rows with NaN or zero/negative x values
    df_clean = df.dropna(subset=['pretraining_prevalence', 'generated_cot_is_correct'])
    df_clean = df_clean[df_clean['pretraining_prevalence'] > 0]
    
    if len(df_clean) < 2:
        return None, None, None
    
    # Prepare x and y values
    if use_log_x:
        x = np.log10(df_clean['pretraining_prevalence'].values)
    else:
        x = df_clean['pretraining_prevalence'].values
    y = df_clean['generated_cot_is_correct'].values
    
    try:
        if fit_type == 'linear':
            # Linear regression
            slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
            params = [slope, intercept]
            
            # Calculate predictions for R²
            y_pred = linear_func(x, *params)
            
        elif fit_type == 'exponential':
            # Exponential fit
            # Initial guess: [amplitude, growth_rate, offset]
            initial_guess = [0.5, 0.1, 0.3]
            bounds = ([0, -np.inf, 0], [1, np.inf, 1])
            params, _ = curve_fit(exponential_func, x, y, p0=initial_guess, 
                                bounds=bounds, maxfev=5000)
            
            # Calculate predictions for R²
            y_pred = exponential_func(x, *params)
            
        elif fit_type == 'sigmoid':
            # Sigmoid fit
            # Initial guess: [amplitude, steepness, x_mid, y_offset]
            x_mid = np.median(x)
            initial_guess = [0.7, 1.0, x_mid, 0.2]
            bounds = ([0, 0, x.min(), 0], [1, 10, x.max(), 1])
            params, _ = curve_fit(sigmoid_func, x, y, p0=initial_guess, 
                                bounds=bounds, maxfev=5000)
            
            # Calculate predictions for R²
            y_pred = sigmoid_func(x, *params)
            
        else:
            raise ValueError(f"Unknown fit_type: {fit_type}")
        
        # Calculate R²
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        r_squared = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        
        return params, r_squared, fit_type
        
    except Exception as e:
        print(f"Error fitting {fit_type}: {e}")
        return None, None, None

# Configuration - CHANGE THESE TO SELECT FIT TYPE
FIT_TYPE = 'linear'  # Options: 'linear', 'exponential', 'sigmoid'
USE_LOG_X = False  # Whether to use log-transformed x for fitting

# Calculate trends for both groups
params_l_algo, r2_l_algo, _ = calculate_fit(df_l_algo, fit_type=FIT_TYPE, use_log_x=USE_LOG_X)
params_rest, r2_rest, _ = calculate_fit(df_rest, fit_type=FIT_TYPE, use_log_x=USE_LOG_X)

# Generate trend line points
x_range = df_viz_tmp_plot['pretraining_prevalence'].values
x_range = x_range[x_range > 0]  # Remove non-positive values
x_range = np.sort(x_range)

if USE_LOG_X:
    x_fit_range = np.log10(x_range)
else:
    x_fit_range = x_range

# Add trend lines if calculations were successful
if params_l_algo is not None:
    if FIT_TYPE == 'linear':
        y_trend_l_algo = linear_func(x_fit_range, *params_l_algo)
    elif FIT_TYPE == 'exponential':
        y_trend_l_algo = exponential_func(x_fit_range, *params_l_algo)
    elif FIT_TYPE == 'sigmoid':
        y_trend_l_algo = sigmoid_func(x_fit_range, *params_l_algo)
    
    fig.add_trace(go.Scatter(
        x=x_range,
        y=y_trend_l_algo,
        mode='lines',
        name=f'Algorithmic encoding, {FIT_TYPE} fit (R²={r2_l_algo:.3f})',
        line=dict(color='red', dash='dash'),
        showlegend=True
    ))

if params_rest is not None:
    if FIT_TYPE == 'linear':
        y_trend_rest = linear_func(x_fit_range, *params_rest)
    elif FIT_TYPE == 'exponential':
        y_trend_rest = exponential_func(x_fit_range, *params_rest)
    elif FIT_TYPE == 'sigmoid':
        y_trend_rest = sigmoid_func(x_fit_range, *params_rest)
    
    fig.add_trace(go.Scatter(
        x=x_range,
        y=y_trend_rest,
        mode='lines',
        name=f'Language encoding, {FIT_TYPE} fit (R²={r2_rest:.3f})',
        line=dict(color='blue', dash='dash'),
        showlegend=True
    ))

# Update layout
fig.update_layout(
    height=600, 
    width=800,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    # title=f"MATH500 Accuracy vs Pretraining Prevalence ({FIT_TYPE.capitalize()} Fit)"
)

fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))

fig.update_yaxes(dtick=0.1, title="MATH500 accuracy", range=[0, 0.6])
fig.update_xaxes(title="Pretraining prevalence", type="log")
fig.update_layout(template="plotly_white")
fig.update_traces(textfont_size=10)

fig.show('png')


# Paper plots

In [ ]:
d_mapping = {
    "baseline": ["zero_shot", "identity"],
    "letter mutation": [
        "reverse_letters_in_each_word",
        "swap_even_odd_letters_in_each_word",
        "reverse_fibonacci_indices_in_each_word",
        "letter_to_word_with_dot",
        "dot_between_chars",
        "space_between_chars",
    ],
    "language deletion": ["remove_all_verbs", "remove_all_nouns"],
    "language translation": [
        "French","Chinese","Korean","Russian","Arabic","Adyghe",
        "Morse_code","Python","enterprise_Java",
    ],
    "algorithmic cipher": [
        "rot13_cipher","base64_cipher","base64_2x_cipher",
        "base64_3x_cipher","caesar_cipher","gzip_to_base64_encoded",
    ],
    "themed reasoning": [
        "paraphrase_naive",
        "pirate_speak",
        "leet_speak",
        "yoda_speak",
        "shakespearean_text",
    ],
    "extraneous content": [
        "insert_tweet",
        "python_snippet_comment",
        "croissant_news_article",
        "math_textbook_article",
        "five_emojis",
    ],
    "delete inf.": [
        "replace_math_content_with_black_box"
    ]
}

l_themed_encodings = [
    "paraphrase_naive",
    "pirate_speak",
    "leet_speak",
    "yoda_speak",
    "shakespearean_text",
    "insert_tweet",
    "python_snippet_comment",
    "croissant_news_article",
    "math_textbook_article",
    "five_emojis",
    "replace_math_content_with_black_box",
    "reverse_letters_in_each_word_no_math_expressions",
    "reverse_letters_in_each_word_only_math_expressions"
]

l_ignore_languages = [
    "Russian",
    "Chinese",
    # "Korean"
]

In [ ]:
import plotly.express as px
import plotly.colors as pc

def sample_colorscale(colorscale_name: str, n: int):
    """
    Sample N equally spaced hex colors from a Plotly continuous colorscale.

    Parameters
    ----------
    colorscale_name : str
        Name of a Plotly built-in continuous colorscale (e.g., "Viridis", "Blues").
    n : int
        Number of samples to return.

    Returns
    -------
    list of str
        List of hex color strings.
    """
    if n < 1:
        raise ValueError("n must be >= 1")
    colorscale = px.colors.get_colorscale(colorscale_name)
    # equally spaced points in [0,1]
    vals = [i/(n-1) if n > 1 else 0.5 for i in range(n)]
    return [pc.sample_colorscale(colorscale, v)[0] for v in vals]

l_gpt_gradient = sample_colorscale("Emrld", 4 + 1)
d_gpt_gradient = {
    model : color
    for model, color in zip(
        ['gpt-4.1-nano-2025-04-14', 'gpt-4.1-mini-2025-04-14', 'gpt-4.1-2025-04-14', 'gpt-5-chat-latest'],
        l_gpt_gradient[1:]
    )
}

l_claude_gradient = sample_colorscale("Magenta", 3 + 1)
d_claude_gradient = {
    model : color
    for model, color in zip(
        ['claude-3-opus-20240229', 'claude-3-5-sonnet-20241022', 'claude-sonnet-4-20250514'],
        l_claude_gradient[1:]
    )
}

l_qwen_gradient = sample_colorscale("Oryel", 3 + 1)
d_qwen_gradient = {
    model : color
    for model, color in zip(
        ['Qwen2.5-3B-Instruct', 'Qwen2.5-7B-Instruct', 'Qwen2.5-14B-Instruct'],
        l_qwen_gradient[1:]
    )
}

In [ ]:
df_viz_tmp[df_viz_tmp['model'].str.contains('gpt')]['encoding_scheme'].unique()

In [ ]:
l_star_encodings = [
    "reverse_letters_in_each_word",
    "rot13_cipher",
    "caesar_cipher", # TODO check
 'reverse_fibonacci_indices_in_each_word', # TODO check
 'swap_even_odd_letters_in_each_word', # TODO check
    "Morse_code"
]

In [ ]:
l_fixed_encoding_ordering = [
    # GPT
    'identity',
    'dot_between_chars',
    'Korean',
    'letter_to_word_with_dot',
    'rot13_cipher',
    'remove_all_verbs',
    'Morse_code',
    'base64_cipher',
    'reverse_letters_in_each_word',

    # placeholder for an empty space
    '',

    # Qwen/Claude
 'French',
 'space_between_chars',
 'Arabic',
 'Adyghe',
 'caesar_cipher',
 'swap_even_odd_letters_in_each_word',
    'reverse_fibonacci_indices_in_each_word',
    'remove_all_nouns',
 'Python',
    'enterprise_Java',
'base64_2x_cipher',
 'base64_3x_cipher',
 'gzip_to_base64_encoded',
 
]

# for i in range(len(l_fixed_encoding_ordering)):
#     if l_fixed_encoding_ordering[i] in l_star_encodings:
#         l_fixed_encoding_ordering[i] += "*"

In [ ]:
# --- Compute PGR (percent of identity performance) ---
df_pgr = df_viz_tmp.copy()

df_pgr = pd.concat([
    df_pgr,
    pd.DataFrame([{'encoding_scheme': '', 'model': model} for model in df_pgr['model'].unique()])
], ignore_index=True)

df_pgr['model'] = df_pgr['model'].str.split('Qwen/').str[-1]

# Identity baseline per model
identity_baseline = (
    df_pgr[df_pgr["encoding_scheme"] == "identity"]
    .set_index("model")["adherent_and_correct"]
)

df_pgr["identity_value"] = np.maximum(df_pgr["model"].map(identity_baseline), 0.0001)

# Avoid divide-by-zero
# df_pgr = df_pgr[df_pgr["identity_value"] > 0].copy()

# Mean PGR as percentage
df_pgr["PGR_pct"] = (df_pgr["adherent_and_correct"] / df_pgr["identity_value"])

# CI deltas -> percentage deltas relative to identity baseline
# low is negative, high is positive
df_pgr["PGR_pct_hi_ci"]  = (df_pgr["adherent_and_correct_hi_ci"] / df_pgr["identity_value"])
df_pgr["PGR_pct_low_ci"] = (df_pgr["adherent_and_correct_low_ci"]        / df_pgr["identity_value"])

df_pgr.loc[df_pgr["encoding_scheme"] == "identity", ["PGR_pct_hi_ci", "PGR_pct_low_ci"]] = 0.0

# order by sonnet 4 hard coded
# df_sonnet4 = df_pgr[df_pgr['model'].str.contains('claude-3-5-sonnet')].sort_values('adherent_and_correct', ascending=False)
# df_sonnet4 = df_pgr[df_pgr['model'].str.contains('14B')].sort_values('adherent_and_correct', ascending=False)
# df_sonnet4 = df_pgr[df_pgr['model'].str.contains('sonnet-4')].sort_values('adherent_and_correct', ascending=False)

# d_encoding_scheme_to_idx = {}
# for i in range(len(df_sonnet4)):
#     d_encoding_scheme_to_idx[df_sonnet4['encoding_scheme'].iloc[i]] = i

# d_encoding_scheme_to_idx['identity'] = 0

for i, encoding in enumerate(l_fixed_encoding_ordering):
    d_encoding_scheme_to_idx[encoding] = i

df_pgr = df_pgr[~df_pgr['encoding_scheme'].isin(l_themed_encodings)]
df_pgr = df_pgr[~df_pgr['encoding_scheme'].isin(l_ignore_languages)]

# df_pgr['encoding_scheme'] = df_pgr['encoding_scheme'].map(lambda x: x + '*' if x in l_star_encodings else x)

df_pgr['sort_order'] = df_pgr['encoding_scheme'].map(d_encoding_scheme_to_idx)
df_pgr = df_pgr.sort_values(['sort_order', 'model_size'])

df_pgr['encoding_scheme'] = df_pgr['encoding_scheme'].map(lambda s: " ".join(s.split('_')))

df_pgr['Model Family'] = df_pgr["model"].map(lambda x: "Qwen2.5" if "Qwen2.5" in x else "GPT" if "gpt" in x else "Claude")

fig = px.line(df_pgr[df_pgr['encoding_scheme'] != 'zero shot'],
              x='encoding_scheme',
              y='PGR_pct',
              color='model',
              color_discrete_map={**d_claude_gradient, **d_gpt_gradient, **d_qwen_gradient},
              markers=True,
              # facet_col="Model Family"
)

# s_encoding_scheme_order = df_pgr[df_pgr['model'].str.contains('14B') & (df_pgr['encoding_scheme'] != 'zero shot')]['encoding_scheme']
# s_encoding_scheme_order = df_pgr[df_pgr['model'].str.contains('sonnet-4') & (df_pgr['encoding_scheme'] != 'zero shot')]['encoding_scheme']
fig.update_layout(
    xaxis=dict(
        categoryorder="array",
        # categoryarray=s_encoding_scheme_order
        categoryarray=[' '.join(s.split('_')) for s in l_fixed_encoding_ordering]
    )
)

fig.update_layout(
    # title=dict(text=title, font=dict(size=22)),
    template="plotly_white",
    height=500,
    width=1800,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.08,
        xanchor="left",
        x=0.01,
        font=dict(size=10)
    ),
    margin=dict(t=110, r=30, b=80, l=70),
    font=dict(size=13),
    hovermode='x unified'
)

fig.update_xaxes(title="Encoding scheme")
fig.update_yaxes(range=[0.0, 1.05], dtick=0.1, title="% of identity adherent & correct")

# add the zero shot lines for 4.1 and 14b
l_zero_shot_models = ['gpt-4.1-2025-04-14', 'Qwen2.5-14B-Instruct']
# l_cols = [1, 1]
l_pos = ["top right", "bottom right"]
l_model_names = ["GPT 4.1", "Qwen2.5 14B"]
# l_zero_shot_models = ['gpt-4.1-2025-04-14', "claude-sonnet-4-20250514"]
# l_cols = [1, 2]

for zero_shot_model, col_idx, pos, model_name in zip(l_zero_shot_models, l_cols, l_pos, l_model_names):
    zero_shot_perf = df_pgr[(df_pgr['model'] == zero_shot_model) & (df_pgr['encoding_scheme'] == 'zero shot')]['PGR_pct'].iloc[0]

    annotation_str = f"{model_name} direct answering perf."
    fig.add_hline(y=zero_shot_perf, line_dash='dash', annotation_text=annotation_str, col=col_idx, annotation_position=pos)

fig.show('png')


In [ ]:
fig = px.line(df_pgr,
              x='encoding_scheme',
              y='backtranslation_bleu_scores',
              color='model',
              color_discrete_map={**d_claude_gradient, **d_gpt_gradient, **d_qwen_gradient},
              markers=True,
              # facet_col="Model Family"
)

# needed for sorting by big model adherent+correct
fig.update_layout(
    xaxis=dict(
        categoryorder="array",
        # categoryarray=s_encoding_scheme_order
        categoryarray=[' '.join(s.split('_')) for s in l_fixed_encoding_ordering]
    )
)

fig.update_layout(
    # title=dict(text=title, font=dict(size=22)),
    template="plotly_white",
    height=500,
    width=1800,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.08,
        xanchor="left",
        x=0.01,
        font=dict(size=10)
    ),
    margin=dict(t=110, r=30, b=80, l=70),
    font=dict(size=13),
    hovermode='x unified'
)

fig.update_xaxes(title="Encoding scheme")
fig.update_yaxes(range=[0, 101], dtick=10, title="BLEU score")

fig.add_hrect(
    y0=60, y1=100,
    fillcolor="#D3D3D3", opacity=0.2,
    layer="below", line_width=0,
    annotation_text="Fluent decoding",
    annotation_font=dict(color="grey")
)

# fig.update_layout(width=600, height=600, title=None)
fig.update_xaxes(tickangle=45)
fig.update_layout(height=510)

fig.show('png')


# More plots

In [ ]:
import plotly.graph_objects as go

def make_encoding_scheme_bar_plot(
    df,
    y_col="generated_cot_is_correct",
    title="MATH 500 Accuracy",
    y_axis_title="Accuracy",
    x_col="encoding_scheme",
    model_col="model",
    d_mapping=None,
    yaxis_dtick=0.1,            # 10% ticks read cleaner
    show_text=False,            # optionally show % text on bars
    model_order=None,           # lock legend/order of models
    font_family="Inter, Arial", # consistent, modern
    sort_by_col=None,
    sort_agg="max",
    sort_desc=True
):
    # --- 1) DEFAULT SECTION MAP ---
    if d_mapping is None:
        d_mapping = {
            "baseline": [
                "zero_shot",
                "identity"
            ],
            # "letter mutation": [
            "": [
                "reverse_letters_in_each_word",
                "reverse_letters_in_each_word_no_math_expressions",
                "reverse_letters_in_each_word_only_math_expressions",
                "swap_even_odd_letters_in_each_word",
                "reverse_fibonacci_indices_in_each_word",
                "letter_to_word_with_dot",
                "dot_between_chars",
                "space_between_chars",
            ],
            "language deletion": ["remove_all_verbs", "remove_all_nouns"],
            "language translation": [
                "French","Chinese","Korean","Russian","Arabic","Adyghe",
                "Morse_code","Python","enterprise_Java",
            ],
            "algorithmic cipher": [
                "rot13_cipher","base64_cipher","base64_2x_cipher",
                "base64_3x_cipher","caesar_cipher","gzip_to_base64_encoded",
            ],
            # "Wording of reasoning": [
            #     "paraphrase_naive",
            #     "pirate_speak",
            #     "yoda_speak",
            #     "shakespearean_text",
            # ],
            # "Distractors": [
            #     "insert_tweet",
            #     "python_snippet_comment",
            #     "croissant_news_article",
            #     # "math_textbook_article",
            #     # "five_emojis",
            # ],
            # "Language": [
            #     "leet_speak",
            #     "rot13_cipher"
            # ],
            # "Inf. content": [
            #     "replace_math_content_with_black_box"
            # ]
        }

    df_plot = df.copy()

     # ===================== NEW SORTING LOGIC =====================
    # Decide which column drives sorting
    metric_col = sort_by_col or y_col

    # Pick aggregation
    agg_map = {"max": "max", "mean": "mean", "median": "median"}
    agg_fn = agg_map.get(sort_agg, "max")

    # Aggregate per scheme across models on the sorting metric
    if metric_col not in df_plot.columns:
        raise ValueError(f"sort_by_col '{metric_col}' not found in dataframe.")

    sort_metric = (
        df_plot
        .groupby(x_col, as_index=False)[metric_col]
        .agg(agg_fn)
        .rename(columns={metric_col: "sort_value"})
    )
    # =============================================================

    # --- 2) ORDERING BY SECTION, THEN BY MAX(Y) DESC ---
    full_category_order, section_spans = [], []
    present_set = set(df_plot[x_col].unique())
    cursor = 0

    for section_name, schemes in d_mapping.items():
        present = [s for s in schemes if s in present_set]
        if not present:
            continue

        section_df = sort_metric[sort_metric[x_col].isin(present)].copy()
        section_df = section_df.sort_values("sort_value", ascending=not sort_desc)
        ordered = section_df[x_col].tolist()
        if not ordered:
            continue

        start = cursor
        full_category_order.extend(ordered)
        cursor = len(full_category_order)
        end = cursor - 1
        section_spans.append((start, end, section_name))

    if not full_category_order:
        full_category_order = list(df_plot[x_col].unique())
        section_spans = [(0, len(full_category_order) - 1, "All")]

    df_plot[x_col] = pd.Categorical(df_plot[x_col], categories=full_category_order, ordered=True)

    # --- 1) PRESERVE MODEL ORDER FROM DATAFRAME ---
    # Use the order models appear in df instead of sorting alphabetically
    if model_order is None:
        model_order = list(dict.fromkeys(df_plot[model_col]))  # <-- preserves original order
    df_plot[model_col] = pd.Categorical(df_plot[model_col], categories=model_order, ordered=True)

    # --- 3) ERROR BARS (auto-detect) ---
    err_hi_col = f"{y_col}_hi_ci"
    err_lo_col = f"{y_col}_low_ci"
    error_y = err_hi_col if err_hi_col in df_plot.columns else None
    error_y_minus = err_lo_col if err_lo_col in df_plot.columns else None

    def wrap_string(s: str, width: int = 15) -> str:
        """
        Wraps the input string so that each line has at most `width` characters.
        If a word exceeds the width, a '-' and newline are inserted.
    
        Args:
            s (str): Input string to wrap.
            width (int): Maximum number of characters per line. Default is 15.
    
        Returns:
            str: The wrapped string.
        """
        result = ""
        i = 0
    
        while i < len(s):
            # Take a chunk of 'width' characters
            chunk = s[i:i+width]
            # If the chunk is exactly width long and not at the end, add a dash
            if len(chunk) == width and i + width < len(s):
                result += chunk + "<br>"
            else:
                result += chunk
            i += width
    
        return result

    # Label prettifier
    def prettify(s: str) -> str:
        s = s.replace("_", "<br>")
        # return wrap_string(s, 10)
        return s

    # --- 2) DETECT WHETHER Y-VALUES ARE PERCENTAGES OR NUMERICAL ---
    y_min, y_max = df_plot[y_col].min(), df_plot[y_col].max()
    is_percent = y_max <= 1.0 and y_min >= 0.0

    # Set default dtick depending on scale
    if yaxis_dtick is None:
        yaxis_dtick = 0.1 if is_percent else None
    
    # --- 2) BASE FIGURE (no change here, just ensures model order applies)
    fig = px.bar(
        df_plot,
        x=x_col,
        y=y_col,
        color=model_col,
        category_orders={x_col: full_category_order, model_col: model_order},
        barmode="group",
        template="plotly_white",
        # color_discrete_sequence=px.colors.qualitative.Safe,
        color_discrete_map={**d_qwen_gradient, **d_gpt_gradient},
        title=title,
        height=600,
        width=1800,
        error_y=error_y,
        error_y_minus=error_y_minus,
        text=(df_plot[y_col] if show_text else None),
    )

    # Y axis as percent
    if is_percent:
        fig.update_yaxes(
            title=y_axis_title,
            tickformat=".0%",
            tickmode="linear",
            dtick=yaxis_dtick,
            rangemode="tozero"
        )
    else:
        fig.update_yaxes(
            title=y_axis_title,
            tickmode="linear",
            rangemode="tozero",
            dtick=yaxis_dtick
        )

    # X axis: rely on categoryarray for order; set readable tick labels
    fig.update_xaxes(
        title="Encoding scheme",
        ticktext=[f"<b>{prettify(lbl)}</b>" for lbl in full_category_order],
        tickvals=full_category_order,       # use category values, not numeric indices
        tickangle=0
    )

    # Bar spacing and text formatting
    fig.update_traces(
        texttemplate = "%{y:.0%}" if (is_percent and show_text) else "%{y:}" if show_text else None,
        textposition="outside" if show_text else "none",
        # Pass extra fields if you want them in hover (optional)
        customdata=df_plot[[model_col]].values,
    )
    fig.update_layout(
        bargap=0.15,
        bargroupgap=0.05,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.08,
            xanchor="left",
            x=0.01,
            font=dict(size=11)  # ↓ Smaller legend font
        ),
        margin=dict(t=110, r=30, b=80, l=70),
        font=dict(family=font_family, size=15),
        title=dict(font=dict(size=22))
    )
    
    # Make x-axis tick labels smaller
    fig.update_xaxes(
        ticktext=[f"<b>{prettify(lbl)}</b>" for lbl in full_category_order],
        tickvals=full_category_order,
        tickangle=0,
        tickfont=dict(size=11)  # ↓ Smaller x-axis font size
    )

    # --- 5) SECTION BACKGROUNDS + LABELS ---
    N = len(full_category_order)

    # Helper: convert [start,end] (category indices) to x domain [0,1]
    def to_xdomain(idx):
        # position of left edge of category idx:
        return idx / N

    # alternating light bands for sections
    shapes = []
    annotations = []
    for i, (start_idx, end_idx, name) in enumerate(section_spans):
        x0 = to_xdomain(start_idx)
        x1 = to_xdomain(end_idx + 1)
        band = dict(
            type="rect",
            xref="x domain", yref="paper",
            x0=x0, x1=x1, y0=0, y1=1,
            layer="below",
            line=dict(width=0),
            fillcolor="rgba(0,0,0,0.03)" if i % 2 == 0 else "rgba(0,0,0,0.00)"
        )
        shapes.append(band)
        annotations.append(dict(
            x=(x0 + x1) / 2, xref="x domain",
            y=1.06, yref="paper",
            text=name, showarrow=False,
            font=dict(size=13, color="black"),
            xanchor="center"
        ))
    fig.update_layout(shapes=shapes)
    for a in annotations:
        fig.add_annotation(**a)

    return fig

In [ ]:
fig = make_encoding_scheme_bar_plot(
    # df=df_pgr,
    df=df_viz_tmp,#[df_viz_tmp['encoding_scheme'] == 'letter_to_word_with_dot'],
    y_col="adherent_and_correct",
    # y_col="PGR_pct",
    title="MATH 500 Accuracy",
    y_axis_title="Accuracy",
    yaxis_dtick=0.1,
    sort_by_col="adherent_and_correct"
)
# fig.update_yaxes(range=[0, 1])

fig.show('png')

In [ ]:
fig = make_encoding_scheme_bar_plot(
    df=df_viz_tmp[df_viz_tmp['model'] == 'gpt-4.1-2025-04-14'],
    y_col="adherent_and_correct",
    # title="Proportion of encoding adherent & correct responses on MATH 500",
    title=None,
    y_axis_title="% of responses adherent & correct",
    yaxis_dtick=0.05,
    sort_by_col="adherent_and_correct"

)

fig.update_yaxes(range=[0, 0.21], dtick=0.03)
fig.update_layout(width=800, height=600)

fig.show('png')

In [ ]:
# --- Compute PGR (percent of identity performance) ---
df_pgr = df_viz_tmp.copy()

# df_pgr = df_pgr[~( (df_pgr['input_type'] == 'mathcot_fewshot') & (df_pgr['encoding_scheme'] == 'identity') )]


df_pgr['model'] = df_pgr['model'].str.split('Qwen/').str[-1]

# df_pgr = df_pgr[~df_pgr['model'].str.contains('gpt')]
# df_pgr = df_pgr[df_pgr['model'].str.contains('14B')]

# Identity baseline per model
identity_baseline = (
    df_pgr[df_pgr["encoding_scheme"] == "identity"]
    .set_index("model")["adherent_and_correct"]
)

df_pgr["identity_value"] = np.maximum(df_pgr["model"].map(identity_baseline), 0.0001)

# Avoid divide-by-zero
# df_pgr = df_pgr[df_pgr["identity_value"] > 0].copy()

# Mean PGR as percentage
df_pgr["PGR_pct"] = (df_pgr["adherent_and_correct"] / df_pgr["identity_value"])

# CI deltas -> percentage deltas relative to identity baseline
# low is negative, high is positive
df_pgr["PGR_pct_hi_ci"]  = (df_pgr["adherent_and_correct_hi_ci"] / df_pgr["identity_value"])
df_pgr["PGR_pct_low_ci"] = (df_pgr["adherent_and_correct_low_ci"]        / df_pgr["identity_value"])

df_pgr.loc[df_pgr["encoding_scheme"] == "identity", ["PGR_pct_hi_ci", "PGR_pct_low_ci"]] = 0.0

fig = make_encoding_scheme_bar_plot(
    df=df_pgr,
    # df=df_pgr[df_pgr['model'] == 'gpt-4.1-2025-04-14'],
    y_col="PGR_pct",
    # title="Relative % of responses adherent & correct vs. identity encoding",
    title=None,
    y_axis_title="% of identity adherent & correct",
    yaxis_dtick=0.1,
    sort_by_col="adherent_and_correct"
)

# fig.update_yaxes(range=[0.0, 1.05])
# fig.update_yaxes(range=[0.0, 0.35], dtick=0.05)
# fig.update_layout(height=475)
# fig.update_traces(width=0.5)
# fig.update_layout(width=800, height=600)


fig.show('png')


In [ ]:
fig = make_encoding_scheme_bar_plot(
    df=df_viz_tmp,
    y_col="cot_gt_logprobs",
     title="Log loss on MATH 500 ground-truth encoded CoT from PRM800K",
    y_axis_title="Mean log loss per sequence",
    yaxis_dtick=250,
    sort_by_col="adherent_and_correct"
)
fig.show()

In [ ]:
df_viz_tmp_plot = df_viz_tmp.copy()

df_viz_tmp_plot = df_viz_tmp_plot[df_viz_tmp_plot['encoding_scheme'] != 'zero_shot']

fig = make_encoding_scheme_bar_plot(
    df=df_viz_tmp_plot,
    y_col="backtranslation_bleu_scores",
     title="Encoded -> English translation BLEU",
    y_axis_title="Encoded -> English translation BLEU",
    yaxis_dtick=10,
    sort_by_col="adherent_and_correct"

)
fig.show('png')

In [ ]:
df_viz_tmp_plot = df_viz_tmp.copy()

df_viz_tmp_plot = df_viz_tmp_plot[df_viz_tmp_plot['encoding_scheme'] != 'zero_shot']

fig = make_encoding_scheme_bar_plot(
    df=df_viz_tmp_plot,
    y_col="total_translation_loss",
     title="Encoded -> English translation log loss",
    y_axis_title="Mean log loss per sequence",
    yaxis_dtick=50,
    sort_by_col="adherent_and_correct"

)
fig.show('png')

# Plot the translation ability -> acc curve

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit

def styled_logistic_scatter_faceted(
    df,
    facet_col="model",
    x_col="backtranslation_gt_logprobs",
    y_col="adherent_and_correct",
    title="Adherence vs. Backtranslation Log-Prob",
    x_axis_title="Backtranslation log-prob",
    y_axis_title="Adherent & correct",
    font_family="Inter, Arial",
    marker_size=7,
    opacity=0.6,
    legend_font_size=10,
    x_tick_font_size=10,
    line_width=3,
    r2_digits=3,               # <— precision for R² display
    keep_subplot_title=True,
    max_pow_override=None
):
    def logistic(x, L, k, x0):
        return L / (1 + np.exp(-k * (x - x0)))

    # overall y-scale to decide percent formatting
    y_all = df[y_col].to_numpy(dtype=float)
    is_percent = (np.nanmin(y_all) >= 0) and (np.nanmax(y_all) <= 1.0)

    # facet ordering
    if pd.api.types.is_categorical_dtype(df[facet_col]):
        facet_values = [c for c in df[facet_col].cat.categories if (df[df[facet_col]==c].shape[0] > 0)]
    else:
        d_model_to_size = dict(zip(df['model'], df['model_size']))
        facet_values = sorted(df[facet_col].dropna().unique(), key=lambda x: d_model_to_size.get(x, x))

    print(facet_values)

    n_cols = max(1, len(facet_values))
    fig = make_subplots(
        rows=1, cols=n_cols, shared_yaxes=True, horizontal_spacing=0.06,
        subplot_titles=[f"{facet_col} = {v}" if keep_subplot_title else None for v in facet_values],
    )

    x_positive = df[df[x_col] > 0][x_col]
    min_x, max_x = x_positive.min(), x_positive.max()
    # Create tick values at decade intervals
    min_pow = int(np.floor(np.log2(min_x)))
    max_pow = int(np.ceil(np.log2(max_x)))
    tick_vals = [2 ** p for p in range(min_pow, max_pow + 1)]
    if max_pow_override:
        if tick_vals[-1] > max_pow_override:
            tick_vals[-1] = max_pow_override
    tick_text = [f"{v:g}" for v in tick_vals]

    for i, val in enumerate(facet_values, start=1):
        sub = df[df[facet_col] == val]
        x = sub[x_col].to_numpy(dtype=float)
        y = sub[y_col].to_numpy(dtype=float)

        # points
        fig.add_trace(
            go.Scatter(
                x=x, y=y, mode="markers+text", name=f"Points ({val})",
                marker=dict(size=marker_size, line=dict(width=0)),
                opacity=opacity, showlegend=False,
                # text=sub["encoding_scheme"] + "-" + sub["model"],
                hovertemplate="<b>%{text}</b><extra></extra>",  # <-- Just encoding scheme
            ),
            row=1, col=i
        )

        # choose regressor space
        # linear_on_log_x = True
        # if linear_on_log_x:
        #     valid = (x > 0) & np.isfinite(x) & np.isfinite(y)
        #     X = np.log2(x[valid])
        #     Y = y[valid]
        # else:
        #     valid = np.isfinite(x) & np.isfinite(y)
        #     X = x[valid]
        #     Y = y[valid]

        # if X.size >= 2 and np.nanstd(Y) > 0:
        #     b1, b0 = np.polyfit(X, Y, 1)   # Y = b1*X + b0
        #     # make a smooth line across current x-range
        #     x_line = np.linspace(np.nanmin(x[valid]), np.nanmax(x[valid]), 300)
        #     X_line = np.log2(x_line) if linear_on_log_x else x_line
        #     y_line = b1 * X_line + b0

        #     # R^2 on observed points (in same space used to fit)
        #     y_hat = b1 * X + b0
        #     ss_res = np.nansum((Y - y_hat) ** 2)
        #     ss_tot = np.nansum((Y - np.nanmean(Y)) ** 2)
        #     if ss_tot > 0 and np.isfinite(ss_res):
        #         r2 = 1 - ss_res / ss_tot
        #         r2_text = f"{r2:.{r2_digits}f}"

        #     fig.add_trace(
        #         go.Scatter(
        #             x=x_line, y=y_line, mode="lines",
        #             name=f"Linear fit ({val})",
        #             # line=linear_line_kwargs,
        #             showlegend=(i == 1),
        #         ),
        #         row=1, col=i
        #     )
        
        # logistic fit + R^2
        r2_text = "—"
        try:
            if np.isfinite(x).sum() >= 3 and np.nanstd(y) > 0:
                p0 = [np.nanmax(y), 1.0, np.nanmedian(x)]
                params, _ = curve_fit(logistic, x, y, p0=p0, maxfev=10000)
                L, k, x0 = params

                x_fit = np.linspace(np.nanmin(x), max(np.nanmax(x), 98), 300)
                y_fit = logistic(x_fit, L, k, x0)

                # R^2 on observed x
                y_hat = logistic(x, L, k, x0)
                ss_res = np.nansum((y - y_hat) ** 2)
                ss_tot = np.nansum((y - np.nanmean(y)) ** 2)
                if ss_tot > 0 and np.isfinite(ss_res):
                    r2 = 1 - ss_res / ss_tot
                    r2_text = f"{r2:.{r2_digits}f}"

                fig.add_trace(
                    go.Scatter(
                        x=x_fit, y=y_fit, mode="lines",
                        name=f"Logistic fit ({val})",
                        line=dict(width=1, color="black", dash="dash"),
                        showlegend=(i == 1),
                    ),
                    row=1, col=i
                )
        except Exception as e:
            print(e)
            pass

        # R^2 annotation (top-left of this subplot)
        fig.add_annotation(
            x=0.1, y=0.55,
            xref=f"x{i}" if i > 1 else "x",   # ✅ Use proper axis names
            yref=f"y{i}" if i > 1 else "y",
            xanchor="left",
            yanchor="top",
            text=f"R² = {r2_text}",
            showarrow=False,
            font=dict(size=12),
            align="left",
            bgcolor="rgba(255,255,255,0.6)",
            bordercolor="rgba(0,0,0,0.2)",
            borderwidth=1,
        )

        # axes cosmetics
        fig.update_xaxes(
            title=x_axis_title if i == 1 else "",
            tickfont=dict(size=x_tick_font_size),
            zeroline=False,
            type="log",
            # autorange="reversed",
            tickvals=tick_vals,
            ticktext=tick_text,
            row=1, col=i,
            dtick=0.1
        )
        fig.update_yaxes(
            tickformat=".0%" if is_percent else ",",
            rangemode="tozero",
            zeroline=False,
            dtick=0.05,
            row=1, col=i,
            title=y_axis_title
        )

    fig.update_layout(
        template="plotly_white",
        title=dict(text=title, font=dict(size=18)),
        height=600,
        width=350 * n_cols if n_cols <= 3 else 300 * n_cols,
        font=dict(family=font_family, size=14),
        margin=dict(t=0, r=0, b=0, l=0),
        legend=dict(orientation="h", yanchor="bottom", y=1.08, xanchor="left", x=0,
                    font=dict(size=legend_font_size)),
        showlegend=False
    )
    # global axis labels
    # fig.add_annotation(text=x_axis_title, showarrow=False, xref="paper", yref="paper",
    #                    x=0.5, y=-0.18, font=dict(size=14))
    # fig.add_annotation(text=y_axis_title, showarrow=False, xref="paper", yref="paper",
    #                    x=-0.06, y=0.5, textangle=-90, font=dict(size=14))

    return fig


# ---- Use it on your dataframe ----
df_viz_tmp_plot = df_viz_tmp.copy()

start_len = len(df_viz_tmp_plot)
df_viz_tmp_plot = df_viz_tmp_plot[~df_viz_tmp_plot['encoding_scheme'].isin(['identity', 'zero_shot'])]
assert len(df_viz_tmp_plot) != start_len

df_viz_tmp_plot["placeholder_col"] = "1"

d_zero_shot_baseline =  (
    df_pgr[df_pgr["encoding_scheme"] == "zero_shot"]
    .set_index("model")["adherent_and_correct"]
)

fig = styled_logistic_scatter_faceted(
    # df_viz_tmp_plot[df_viz_tmp_plot['backtranslation_bleu_scores'].notna() & ~df_viz_tmp_plot['model'].str.contains('gpt')],
    #[df_viz_tmp_plot['adherent_and_correct'] > 0.01],
    df_viz_tmp_plot,
    # df_viz_tmp_plot[(df_viz_tmp_plot['adherent_and_correct'] >= df_viz_tmp_plot['model'].map(d_zero_shot_baseline))],
    # x_col="backtranslation_gt_logprobs",
    x_col="backtranslation_bleu_scores",
    y_col="adherent_and_correct",
    # y_col='cot_gt_logprobs',
    # title="% of responses adherent & correct vs. encoded -> English translation BLEU",
    title=None,
    x_axis_title="BLEU score, decoding encoded text to English",
    # x_axis_title="Encoded text to English text log loss",
    y_axis_title="% of MATH500 responses adherent & correct",
    # facet_col="model",
    facet_col="placeholder_col",
    keep_subplot_title=False,
    max_pow_override=100,
    x_tick_font_size=12
)


fig.update_layout(width=600, height=500, title=None)
# fig.update_layout(width=1600, height=1200, title=None)
# --- shaded region instead of vline ---
x_cut = 60
x_right = float(df_viz_tmp_plot["backtranslation_bleu_scores"].max()) + 30
# x_cut = 0.2
# x_right = 0.02


fig.add_vrect(
    x0=x_cut, x1=x_right,           # shade from cutoff to the right edge of data
    fillcolor="#D3D3D3", opacity=0.25,
    line_width=0,
    layer="below",
    row=1, col=1                    # adjust if you facet into multiple columns
)

# label for the shaded region
fig.add_annotation(
    x=0.84,
    # x=0.85,
    xref="paper",           # data coordinates
    y=0.99, yref="paper",
    text="fluent translation",
    showarrow=False,
    xanchor="left",
    yanchor="top",
    bgcolor="rgba(255,255,255,0.0)",
    bordercolor="rgba(0,0,0,0.0)",
    borderwidth=0,
    font=dict(size=9)
)
fig.update_traces(textposition="bottom left")


fig.show('png')
# fig.show()


In [ ]:
l_test_hashes = ['f01404b75a072276ac1ca6f131313a8a615dbb99', '481e3a2a1f521438c8938721b0df08a389e3406f', 'b411a66e1c2f9ca7a79c065099879f202990c9a2', '33c2595336eb7c74c6820302498fd097fa52463c', 'e3f76d94751a09af21005696ac072219143ebc58', 'b77d4d2334cc5dad12f2168d5fa48749622f34af', '2007b98cc05baeb988564f2e51d6f496a8afe7b8', 'f0a6d8673396c2f4415e10c2fc13effa262fe50d', '16b151e08633744e1c501d6f41707177c5aa6125', 'a846c2161731fabfe9b9d371ac05473c5ba9289d', 'a9a752f3c050a16ef74c6ad9436f0aa6291fb1ff', 'b4c85c27bcc89f916f50867f974bd13875416067']

for hash in l_test_hashes:
    target_path = f"/home/ubuntu/sky_workdir/encoding-schemes/output/{hash}/data/sft_train.parquet"
    if os.path.exists(target_path):
        print(duckdb.query(f"SELECT SUM(num_tokens) FROM read_parquet('{target_path}')").to_df())

# Inspect the data

In [ ]:
from encoding_schemes import get_deterministic_adherence_fn
from encoding_schemes.letter_permutations import get_English_dictionary, reverse_letters_in_each_word, normalize_word

In [ ]:
test_scheme = 'speaking_letter_to_word_with_dot'
test_model = 'Qwen/Qwen2.5-14B-Instruct'
test_idx = 21

adherence_fn = get_deterministic_adherence_fn(test_scheme, None)

In [ ]:
df_viz[(df_viz['model'] == test_model) & (df_viz['encoding_scheme'] == test_scheme)]['reference_problem_df'].iloc[0][test_idx]

In [ ]:
df_viz[(df_viz['model'] == test_model) & (df_viz['encoding_scheme'] == test_scheme)]['reference_solution_df'].iloc[0][test_idx]

In [ ]:
df_viz[(df_viz['model'] == test_model) & (df_viz['encoding_scheme'] == test_scheme)]['generated_cots_df'].iloc[0][test_idx][0]

In [ ]:
reverse_letters_in_each_word(df_viz[(df_viz['model'] == test_model) & (df_viz['encoding_scheme'] == test_scheme)]['generated_cots_df'].iloc[0][test_idx][0])

In [ ]:
df_viz[(df_viz['model'] == test_model) & (df_viz['encoding_scheme'] == test_scheme)]['generated_cot_adhered_encoding_style'].iloc[0]#[test_idx]

In [ ]:
example_hash = df_viz[(df_viz['model'] == test_model) & (df_viz['encoding_scheme'] == test_scheme)]['experiment_hash'].iloc[0]
example_hash

In [ ]:
with open(f"/home/ubuntu/sky_workdir/encoding-schemes/output/{example_hash}/data/sft_model_meta.json", "r") as fp:
    d_example = json.load(fp)

d_example

In [ ]:
sft_test_path = f"/home/ubuntu/sky_workdir/encoding-schemes/output/{example_hash}/data/sft_train.parquet"

df_sft = pd.read_parquet(sft_test_path)
df_sft['messages'].iloc[0]

In [ ]:
len(df_sft)

In [ ]:
df_sft = df_sft.iloc[:3000]

In [ ]:
df_sft['num_tokens'].sum()

In [ ]:
def convert_sft_parquet_to_jsonl(df, output_json_path):
    n_rows_written = 0
    with open(output_json_path, "w", encoding="utf-8") as f:
        for idx, raw in enumerate(df["messages"]):

            json_line = {"messages": list(raw)}
            f.write(json.dumps(json_line, ensure_ascii=False) + "\n")
            n_rows_written += 1

    print(f"[prep] Wrote {n_rows_written} training rows to {output_json_path}")

In [ ]:
convert_sft_parquet_to_jsonl(df_sft, sft_test_path.replace("parquet", "jsonl"))

In [ ]:
df_sft = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/e4a87a8626efeeb6df76d94bb34e7e5e34c77154/data/sft_train.parquet")

df_sft = df_sft.iloc[:9120]

df_sft.head()

In [ ]:
df_sft['num_tokens'].sum()